In [ ]:
# Read the csv and create excel file from it


In [ ]:
# Absolute path to this file
MY_PAPI_DIR = pathlib.Path().absolute()
# Now, we have to move to the root of this workspace ([prev. path]/TFG)
MY_PAPI_DIR = MY_PAPI_DIR.parent.parent.parent.parent.absolute()
# From the root (TFG/) access to my_papi dir. and its content
MY_PAPI_DIR = MY_PAPI_DIR / "my_papi"
# Folder where the configuration files are located
CFG_DIR = MY_PAPI_DIR / "conf"
# Folder where the library is located
LIB_DIR = MY_PAPI_DIR / "lib"
# Folder where the source codes are located
SRC_DIR = MY_PAPI_DIR / "src"

# Add the source path and import the library
sys.path.insert(0, str(SRC_DIR))
from MyPapi import *

In [ ]:
def plot_each_iter(csv_file):

    # Read csv with the following name of columns
    df = pd.read_csv(csv_file, header=None, sep=":",
                        names=["CPU", "Value", "Unit", "Event Name"])

    # Get the events and cpus measured
    events = df["Event Name"].unique()
    cpus = df["CPU"].unique()

    # Also the number of iterations (batch_size, epoch, etc)
    num_measures = int(len(df.index) / (len(events) * len(cpus)))

    # Creates a column with the number of iteration
    df.insert(0, "# Measure", 0)

    # We have to modify them depending on the number of measures and cpus
    aux = 0
    for i in range(num_measures, 0, -1):
        aux += 1
        df.loc[df.index[-i * len(events) * len(cpus):], "# Measure"] = aux

    # "Rotate" the table
    df = df.pivot_table(index=["# Measure", "CPU"], columns=[
        "Event Name"], values=["Value"]).fillna(0)

    # Drop the first multiindex
    df.columns = df.columns.droplevel()

    # Add columns with rates (IPC, acc., etc.)
    df = MyPapi.get_rates_from_df(df)

    # Remove name of columns
    df.columns.name = None
    # Reset the index to an auto-increment
    df = df.reset_index()

    # No need of CPU column
    df = df.drop(["CPU"], axis=1)

    # Get the list of measures
    measures = df["# Measure"].unique()
    events = df.columns

    # Array with 'num_measures' dicts as entries
    arr_measures = [{} for _ in measures]

    # Store the data
    for i in range(0, len(measures)):
        for e in events:
            dict_aux = arr_measures[i]
            dict_aux[e] = df.loc[df["# Measure"] == measures[i], e].mean()

    # Convert to pandas Dataframe
    df = pd.DataFrame(arr_measures)
    # Set the # Measure column as index
    df = df.set_index("# Measure")

    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    # Array with the figures/html files to create
    figs = [go.Figure(),
            make_subplots(
                rows=1, cols=2,
                specs=[[{"secondary_y": False}, {"secondary_y": False}]],
                subplot_titles=("Rates", "Derived metrics"))
            ]

    # Add the value to the graphs
    for col in df.columns:
        if col not in events_dict.keys():
            figs[0].add_trace(
                go.Scatter(x=df.index.values.tolist(), y=df[col], name=col)
            )
        else:
            if "rate" in col:
                figs[1].add_trace(
                    go.Scatter(x=df.index.values.tolist(),
                                y=df[col] * 100, name=col),
                    row=1, col=1, secondary_y=False
                )
            else:
                figs[1].add_trace(
                    go.Scatter(x=df.index.values.tolist(),
                                y=df[col], name=col),
                    row=1, col=2, secondary_y=False
                )
    # Change the name of the y axis
    figs[0].update_yaxes(title_text="Value")
    figs[1].update_yaxes(title_text="Miss rate (%)", row=1, col=1)
    figs[1].update_yaxes(title_text="Value", row=1, col=2)
            
    # Set options common to all traces with fig.update_traces
    for fig in figs:
        fig.update_traces(mode='lines+markers',
                            marker_line_width=2, marker_size=8)
        fig.update_layout(
            title='MyPaPi measure by iterations: ' + csv_file,
            # yaxis_zeroline=False, xaxis_zeroline=False,
            hovermode="x unified",
            legend=dict(
                # x=-1,
                # y=-1,
                traceorder="normal",
                font=dict(family="sans-serif",
                            size=12,
                            color="black"),
                bgcolor="white",
                bordercolor="Black",
                borderwidth=2
            )
        )
        fig.update_xaxes(range=[0, len(measures) + 1],
                            title_text="Number of measure")
        # ! Open the files when end the execution
        #fig.show()

    # Save the html files
    name_html = csv_file + "_1_.html"
    figs[0].write_html(name_html)